## Empirics
#### 11.0 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-02-18

## Setup

In [ ]:
from algosdk.v2client import algod
from algosdk import mnemonic
from algosdk import transaction
from algosdk.v2client import indexer
import json
import requests
import pandas as pd

In [ ]:
from pyteal import *

In [ ]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

In [ ]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

## Transaction types
There are six types of transaction (only the first three are relevant):

| Name                | json resp |
|---------------------|-----------|
| Payment             | "pay"     |
| Asset Transfer      | "axfer"   |
| Asset Configuration | "acfg"    |
| Asset Freeze        | "afrz"    |
| Application Call    | "appl"    |
| Key Registration    | "keyreg"  |

Ref: https://developer.algorand.org/docs/get-details/transactions/


### Example step-by-step

In [ ]:
url = 'https://mainnet-algorand.api.purestake.io/idx2/v2/blocks/18946778'

In [ ]:
# typical structure if dict used to make request
# we can use cred['purestake_token'] that returns a dictionary itself

r = requests.get(url = url, headers = cred['purestake_token'])
data = r.json()

In [ ]:
# If you are curious, uncomment this line
data

In [ ]:
len(data['transactions']) # this is the number of all transaction in a block 

In [ ]:
tx_type = {
    'names' : ['Payment', 'Key Registration', 'Asset Configuration', 'Asset Freeze', 'Asset Transfer', 'Application Call'],
    'json_resp' : ['pay', 'keyreg', 'acfg', 'afrz', 'axfer', 'appl']
}

In [ ]:
app = [x for x in data['transactions'] if x['tx-type'] == 'appl']
payments = [x for x in data['transactions'] if x['tx-type'] == 'pay']

In [ ]:
print(len(app))
print(len(payments))

In [ ]:
for k in tx_type['json_resp']:
    list_ = [x for x in data['transactions'] if x['tx-type'] == k]
    print('In this block there are {} {} transactions.'.format(len(list_), tx_type['names'][tx_type['json_resp'].index(k)]))

### Utility function for analyzing one block
* Reads a block
* Returns a dataframe

In [ ]:
def tx_given_block(block_num, purestake_token):
    
    '''returns a DF containing the number of all 6 txs types given a block number.
     a private key must be passed as argument'''
    
    df_block = pd.DataFrame()
    df_block['Block_number'] = [block_num]
    url = 'https://mainnet-algorand.api.purestake.io/idx2/v2/blocks/{}'.format(int(block_num))
    r = requests.get(url = url, headers = purestake_token)
    data = r.json()
    
    tx_type = {
    'names' :     ['Payment', 'Key Registration', 'Asset Configuration', 'Asset Freeze', 'Asset Transfer', 'Application Call'],
    'json_resp' : ['pay', 'keyreg', 'acfg', 'afrz', 'axfer', 'appl'] }
    
    for k in tx_type['json_resp']:
        
        list_ = [x for x in data['transactions'] if x['tx-type'] == k]
        tx_name = tx_type['names'][tx_type['json_resp'].index(k)]
        df_block[str(tx_name)] = [len(list_)]
        
    df_block['Total_txs'] = len(data['transactions'])
    
    return df_block

In [ ]:
tx_given_block(18946779, cred['purestake_token'])

### Read 50 blocks starting from last block
* Takes approx 25sec

In [ ]:
result = pd.DataFrame()

last_block = algod_client.status()["last-round"]
print(f"Last committed block is: {last_block}")

starting_block = last_block
ending_block = last_block-50

for k in range(starting_block, ending_block):
    result = result.append(tx_given_block(k, cred['purestake_token']))

In [ ]:
# Drop the block number from dataframe
result.drop('Block_number', axis= 1).mean()

In [ ]:
# Simplified plot from Pandas
result.drop('Block_number', axis = 1).mean().plot.bar()

## Assets in blocks

- get all the transaction in a given block 
- divide them by asset
- plot the shares of assets ( highlight USDT e ALGO ) 

### Example step-by-step

In [ ]:
url = 'https://mainnet-algorand.api.purestake.io/idx2/v2/blocks/18946778'

In [ ]:
r = requests.get(url = url, headers = cred['purestake_token'])
data = r.json()

In [ ]:
all_txs_list = [x for x in data['transactions'] if x['tx-type'] == 'axfer']

In [ ]:
# to understand the structure of the obkect data['transactions']
all_txs_list[0]['asset-transfer-transaction']['asset-id']

In [ ]:
asset_IDs = [all_txs_list[k]['asset-transfer-transaction']['asset-id'] for k in range(0,len(all_txs_list)) ]

In [ ]:
# A lot of different assets. most of them with only 1 transaction (address that later)
len(asset_IDs)

### Utility function for analzing assets in a block
- The function creates a dataframe usable along the code (e.g., loop)

In [ ]:
def assets_given_block(block_num, purestake_token):
    
    '''returns a DF containing the shares of assets given a block number.
     a private key must be passed as argument'''
    
    df_block = pd.DataFrame()
    df_block['Block_number'] = [block_num]
    url = 'https://mainnet-algorand.api.purestake.io/idx2/v2/blocks/{}'.format(int(block_num))
    r = requests.get(url = url, headers = purestake_token)
    data = r.json()
    
    # ALGOs
    all_txs_list = [x for x in data['transactions'] if x['tx-type'] == 'pay']
    df_block['ALGO'] = len(all_txs_list)
    
    ## ASA 
    #create a list with all transaction "asset transfer" axfer
    all_txs_list = [x for x in data['transactions'] if x['tx-type'] == 'axfer']
    # Create a list with all assets ID
    asset_IDs = [all_txs_list[k]['asset-transfer-transaction']['asset-id'] for k in range(0,len(all_txs_list)) ]
    for k in set(asset_IDs):
        df_block[str(k)] = asset_IDs.count(k)
    

    return df_block
    

In [ ]:
assets_given_block(18946778, cred['purestake_token'])

In [ ]:
result = pd.DataFrame()

starting_block = 18946778
ending_block = starting_block+10

for k in range(starting_block, ending_block):
    
    result = result.append(assets_given_block(k, cred['purestake_token']))

In [ ]:
# Drop the block number since there is no sense in computing the mean of these datapoints
small_df = result.drop('Block_number', axis = 1).mean()
#I want to plot all the asset id that are used more than once in the whole interval. 
small_df = small_df[small_df > 1]
print(len(small_df))
small_df.plot.bar()

### Improvements
- get a list of asset-id vs asset name -> ask
- can I use matplotlib to change the x-label with the names of assets? or we just say that if someone want to know the name of the asset can use the function?

#### List of assets from ID

In [ ]:
url = 'https://mainnet-algorand.api.purestake.io/idx2/v2/assets/27165954'

In [ ]:
r = requests.get(url = url, headers = cred['purestake_token'])
data = r.json()

### Util function to obtain asset names
- Inputs one asset ID and purestake API key
- Returns dict with asset info

In [ ]:
def name_asset_id(asset_id, purestake_token):
    
    '''returns a dictionary containing the name and the unit-name of assets given asset-id.
     a private key must be passed as argument'''
    
    url = 'https://mainnet-algorand.api.purestake.io/idx2/v2/assets/{}'.format(int(asset_id))
    
   
    r = requests.get(url = url, headers = purestake_token)
    data = r.json()
    
    my_dict = {
            'asset_id': asset_id,
            'name': data['asset']['params']['name'],
            'unit-name': data['asset']['params']['unit-name']
            }
    return my_dict

In [ ]:
name_asset_id(27165954,cred['purestake_token'])

In [ ]:
# Get the names of the assets in the DF
# Starts from 2 because we know the first is always ALGO and then there is the block number

# This takes 10/12 seconds
names = []
for k in result.columns[2:]:
    names.append(name_asset_id(k,cred['purestake_token'])['unit-name'])

In [ ]:
# If you are interested uncomment to see every name.
# Names

In [ ]:
plot_df = pd.DataFrame()

In [ ]:
plot_df['names'] = ['ALGO'] + names

In [ ]:
plot_df['average'] = list(result.drop('Block_number', axis = 1).mean())

In [ ]:
plot_df[plot_df['average'] >1].plot.bar(x = 'names', y = 'average')